# Machine Learning Asset Allocation

### Loading Libraries

In [2]:
# Randomness
import random

# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
from pandas import Timestamp

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objects as go
import plotly.io as pio
%matplotlib inline

# Date & Time
from datetime import datetime, timedelta

# Typing
from typing import Tuple, List, Dict, Union, Optional, Any, Generator

# Scikit-Learn
from sklearn.pipeline import Pipeline
from sklearn.metrics import RocCurveDisplay
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection._split import _BaseKFold
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss

# Scientific Statistical Python
import scipy.cluster.hierarchy as sch
from scipy.stats import jarque_bera
from scipy.stats import rv_continuous, kstest, norm

#### Inverse Invariance Portafolio

In [3]:
def get_ivp(cov: np.ndarray, **kargs) -> np.ndarray:
    ivp = 1.0 / np.diag(cov)
    ivp /= ivp.sum()
    return ivp

#### Variance per Cluster

In [4]:
def get_cluster_var(cov: np.ndarray, cItems: np.ndarray) -> float:
    cov_ = cov.loc[cItems, cItems]    # matrix slice
    w_ = get_ivp(cov_).reshape(-1, 1)
    cVar = np.dot(np.dot(w_.T, cov_), w_)[0, 0]
    return cVar